In [115]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
% matplotlib inline
pd.set_option("display.max_columns", 30)

df = pd.read_csv('train.csv')
df2 = pd.read_csv('test.csv')


In [116]:
len(df)

1460

In [117]:
len(df2)

1459

In [118]:
y = df["SalePrice"].as_matrix()
df = pd.concat([df,df2])
df.reset_index(drop=True, inplace=True)

In [119]:
len(df)

2919

In [120]:
df.columns

Index(['1stFlrSF', 'BsmtQual', 'ExterQual', 'Fireplaces', 'Foundation',
       'FullBath', 'GarageArea', 'GarageCars', 'GarageFinish', 'GarageType',
       'GarageYrBlt', 'GrLivArea', 'HeatingQC', 'KitchenQual', 'MSSubClass',
       'MasVnrArea', 'MasVnrType', 'Neighborhood', 'OverallQual', 'SalePrice',
       'TotRmsAbvGrd', 'TotalBsmtSF', 'YearBuilt', 'YearRemodAdd'],
      dtype='object')

In [124]:
df.drop(["SalePrice"],axis=1,inplace = True)

In [125]:
#traindataの取り出し
X = df.iloc[:,:][0:1460].values
X.shape

(1460, 23)

In [126]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 0)

In [127]:
import xgboost as xgb
mod = xgb.XGBRegressor()
mod.fit(X_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1)

In [128]:
# MSEを出力する関数を読み込む
from sklearn.metrics import mean_squared_error as mse

# RMSEをtrainとtestに分けて出力
# 過学習をしているかどうかを確認
print('RMSE')
print('train: %.3f' % (mse(np.log(y_train), np.log(mod.predict(X_train)) ** 1/2)))
print('test : %.3f' % (mse(np.log(y_test), np.log(mod.predict(X_test)) ** 1/2)))

RMSE
train: 36.138
test : 36.200


In [129]:
from sklearn.model_selection import GridSearchCV

In [130]:
params = {'max_depth': [3, 5, 10], 'learning_rate': [0.05, 0.1], 'max_depth': [3, 5, 10, 100], 'subsample': [0.8, 0.85, 0.9, 0.95], 'colsample_bytree': [0.5, 1.0]}

In [135]:
mod = xgb.XGBRegressor()
# 10-fold Cross Validationでパラメータ選定
cv = GridSearchCV(mod, params, cv = 5, scoring= 'neg_mean_squared_error', n_jobs =1)
cv.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True, subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': [3, 5, 10, 100], 'learning_rate': [0.05, 0.1], 'subsample': [0.8, 0.85, 0.9, 0.95], 'colsample_bytree': [0.5, 1.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring='neg_mean_squared_error', verbose=0)

In [136]:
print('RMSE')
print('train: %.3f' % (mse(np.log(y_train), np.log(cv.predict(X_train)) ** 1/2)))
print('test : %.3f' % (mse(np.log(y_test), np.log(cv.predict(X_test)) ** 1/2)))

RMSE
train: 36.135
test : 36.183


In [142]:
X1 = df.iloc[:,:][1460:].values
X1.shape

(1459, 23)

In [143]:
y2 = cv.predict(X1)
a =[i for i in range(1461,2920)]
dfnew = pd.DataFrame({'SalePrice':y2,'Id': a})
dfnew.to_csv("pred10.csv",index = False)